In [50]:
import html
import re
from collections import defaultdict
from pathlib import Path

import numpy as np
import pandas as pd

# import nltk  # noqa: ERA001
import pyarrow.csv as pv
from scipy.sparse import dok_matrix

In [2]:
pd.options.future.infer_string = True  # type: ignore[attr-defined]

Optamos por não utilizar o NLTK no final, pois como utilizamos embeddings, podemos acabar perdendo o contexto de certas coisas por conta disso.

Exemplo:
- A frase "não é bom" e "é bom" podem ser tratadas como similares caso a palavra "não" acabe sendo removida
- Se o embedding já consegue entender o contexto em um certo grau, a lematização também pode ser dispensada

In [ ]:
# - from nltk.corpus import stopwords
# - from nltk.stem import RSLPStemmer
# - from nltk.tokenize import word_tokenize
# -> nltk.download("omw-1.4")
# -> nltk.download("wordnet")
# -> nltk.download("averaged_perceptron_tagger")
# -> nltk.download("maxent_ne_chunker")
# -> nltk.download("words")
# -> nltk.download("punkt")
# -> nltk.download("punkt_tab")
# -> nltk.download("stopwords")
# -> nltk.download("rslp")

from unidecode import unidecode

In [60]:
path = "../data/interim/itens_popularity.parquet"

itens = pd.read_parquet(path, engine="pyarrow", dtype_backend="pyarrow")
itens

,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...,['am' 'amazonas' 'noticia'],57,3,10875.0,1.386294,4.280184,0.247584
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém Ádrio Denner/ AD Produ...,As expressões santarenas não significam apenas...,['pa' 'santarem-regiao' 'noticia'],1151,2,15000.0,1.098612,3.845143,0.0
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...,['mundo' 'noticia'],37,12498,60000.0,9.433404,75.467231,11.866254
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...,['politica' 'noticia'],339,3,35856.5,1.386294,7.743355,0.0
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"Após 2 votos, pedido de vista suspende julgam...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic...",['politica' 'noticia'],333,6,41051.25,1.94591,11.880025,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255598,aca76b7b-9c02-4070-a4a9-00f8c9b4dae2,http://g1.globo.com/politica/eleicoes/2022/not...,2022-04-18 23:32:54+00:00,2022-05-02 10:53:35+00:00,"Eleições: título de eleitor cancelado, o que f...",Detalhe da urna eletrônica e a tecla confirma ...,Cidadão pode consultar a situação do título de...,['politica' 'eleicoes-2022' 'noticia'],118,3,50000.0,1.386294,9.704061,0.026584
255599,20de2172-b557-4881-8c55-2706db029e9b,http://g1.globo.com/politica/eleicoes/2022/not...,2022-08-09 00:53:27+00:00,2022-09-19 20:30:25+00:00,Plano de governo: Ciro Gomes (PDT),"Candidato do PDT à Presidência da República, C...","Em documento de 26 páginas, candidato à Presid...",['politica' 'eleicoes-2022' 'noticia'],6,155,47431.0,5.049856,34.051684,25.226108
255600,347c7e3d-ff53-4ba0-8de7-8bb860b04a88,http://g1.globo.com/politica/eleicoes/2022/not...,2022-03-15 18:18:54+00:00,2022-03-15 18:26:34+00:00,Moro vê economia brasileira estagnada e diz qu...,O pré-candidato do Podemos à Presidência da Re...,Ex-ministro da Justiça de Bolsonaro e pré-cand...,['politica' 'eleicoes-2022' 'noticia'],152,1,9409.0,0.693147,2.038477,0.00102
255601,26f7ff70-46da-42d2-9354-578e8f828a16,http://g1.globo.com/politica/eleicoes/2022/not...,2022-03-23 15:13:50+00:00,2022-03-24 00:54:34+00:00,Lula é hoje 'aquele que melhor reflete o senti...,"Ex-governador de SP, Geraldo Alckmin se filia ...",Ex-governador de SP deu a declaração durante a...,['politica' 'eleicoes-2022' 'noticia'],144,4,20027.25,1.609438,6.442137,0.00481


In [5]:
itens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255603 entries, 0 to 255602
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype                         
---  ------                    --------------   -----                         
 0   page                      255603 non-null  large_string[pyarrow]         
 1   url                       255603 non-null  large_string[pyarrow]         
 2   issued                    255603 non-null  timestamp[ns, tz=UTC][pyarrow]
 3   modified                  255603 non-null  timestamp[ns, tz=UTC][pyarrow]
 4   title                     255603 non-null  large_string[pyarrow]         
 5   body                      255603 non-null  large_string[pyarrow]         
 6   caption                   255603 non-null  large_string[pyarrow]         
 7   tags                      255603 non-null  list<element: string>[pyarrow]
 8   days_since_issue          255603 non-null  int64[pyarrow]                
 9   unique_users   

Pré-processamento dos textos

O texto que mais precisa de processamento é o da coluna `body`, que conta com o texto do corpo da página.

Obs.: as notícias com conteúdo interativo tem seu conteúdo textual limitado, mas muitas vezes aparecem de uma forma estranha (ex.: elementos HTML e CSS no meio do texto). Mesmo assim, como é algo que não aparece em um número elevado de notícias, realizar um tratamento extremamente específico não é algo muito adequado.

In [6]:
itens[itens["url"].str.contains("especiais.g1.globo")]

,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay
3297,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/monitor-da-viole...,2018-02-22 07:00:00+00:00,2018-02-22 07:00:00+00:00,Raio X do sistema prisional em 2018,"Mapa mostra número de presos por agente, super...","Mapa mostra número de presos por agente, super...",['monitor-da-violencia-2018' 'raio-x-do-sistem...,1634,3,20000.0,1.386294,5.545177,0.0
3298,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/economia/2016/ca...,2016-12-10 08:04:41+00:00,2016-12-10 08:04:41+00:00,Calculadora: Veja quando você se aposentará co...,notícias esportes entretenimento vídeos ...,Ferramenta simula quando você se aposentará se...,['economia-2016' 'calculadora-nova-regra-apose...,2073,11,43846.0,2.484907,15.865135,0.0
9822,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/bahia/2017/as-pr...,2017-01-01 07:00:00+00:00,2022-12-12 18:25:43+00:00,As promessas de ACM Neto,ACM Neto fez promessas específicas em um progr...,ACM Neto fez promessas específicas em um progr...,['bahia-2017' 'as-promessas-de-acm-neto'],2051,10,40000.0,2.397895,14.387372,0.0
9823,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/parana/2017/as-p...,2017-01-01 07:00:00+00:00,2022-12-12 18:39:55+00:00,As promessas de Greca,Greca fez promessas específicas em um programa...,Greca fez promessas específicas em um programa...,['parana-2017' 'as-promessas-de-greca'],2051,1,20176.0,0.693147,2.784788,0.0
9824,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/amazonas/2017/as...,2017-01-01 07:00:00+00:00,2022-12-12 18:29:07+00:00,As promessas de Arthur,Arthur fez promessas específicas em um program...,Arthur fez promessas específicas em um program...,['amazonas-2017' 'as-promessas-de-arthur'],2051,1,57990.0,0.693147,5.405855,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250566,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/parana/2017/as-p...,2017-01-01 07:00:00+00:00,2020-01-08 12:12:02+00:00,As promessas de Greca,Greca fez promessas específicas em um programa...,Greca fez promessas específicas em um programa...,['parana-2017' 'as-promessas-de-greca'],2051,11,13725.0,2.484907,8.380348,0.0
250568,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/maranhao/2015/as...,2015-12-29 07:00:00+00:00,2018-12-28 19:41:48+00:00,As promessas de Dino,Dino fez promessas específicas em um programa ...,Dino fez promessas específicas em um programa ...,['maranhao-2015' 'as-promessas-de-dino'],2420,11,39841.0,2.484907,14.86993,0.0
250572,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/rio-de-janeiro/2...,2017-08-26 20:04:41+00:00,2017-09-22 20:04:41+00:00,PMs mortos no RJ,notícias esportes entretenimento vídeos ...,Especial do G1 mostra os policiais militares q...,['rio-de-janeiro-2017' 'pms-mortos-no-rj'],1814,20,50000.0,3.044522,21.311657,0.0
250575,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/educacao/guia-de...,2017-08-09 07:00:00+00:00,2017-08-09 07:00:00+00:00,Teste Vocacional - Faça e encontre o seu perfi...,notícias esportes entretenimento vídeo...,Detalhe o seu perfil no questionário e o G1 in...,['educacao' 'guia-de-carreiras-2017' 'teste-vo...,1831,90,14848.0,4.51086,15.719443,0.0


Tratando os textos:

In [7]:
with pd.option_context("display.max_colwidth", 1):
    display(itens["body"][0])

'Após audiência de custódia, a Justiça do Amazonas decretou, na tarde deste sábado (18), a prisão temporária,  por 30 dias, de Jeferson da Silva Lima, conhecido como "Pelado da Dinha". Ele teve participação direta na morte do indigenista Bruno Pereira e do jornalista inglês Dom Phillips, aponta as investigações. \n"Pelado da Dinha" foi considerado foragido na noite de sexta-feira (17) após ter o mandado de prisão expedido e não ser localizado pelas autoridades. Ele se entregou na delegacia de Atalaia do Norte, a 1.136 quilômetros de Manaus, nas primeiras horas da manhã deste sábado, onde foi ouvido pelo delegado Alex Perez Timóteo. \n"Pelado da Dinha" foi considerado foragido na noite de sexta-feira (17) após ter o mandado de prisão expedido \nRôney Elias/Rede Amazônica \nDurante a tarde, Jeferson foi escoltado por agentes da Polícia Federal ao Fórum de Justiça do município para a audiência de custódia e teve a prisão temporária decretada. \nPerícia confirma identificação dos restos mo

In [9]:
def preprocess_text(text: str) -> str:
    """Preprocess text."""
    if pd.isna(text):
        display("\nHá texto(s) vazio(s)")
        return ""

    text = re.sub(r"Compartilhe esta notícia no \w+", "", text, flags=re.IGNORECASE)
    text = re.sub(r"Compartilhe no \w+", "", text, flags=re.IGNORECASE)
    text = re.sub(r"Reprodução/\w+", "", text, flags=re.IGNORECASE)
    text = re.sub(
        r"VÍDEOS: Mais assistidos do g1 nos últimos 7 dias",
        "",
        text,
        flags=re.IGNORECASE,
    )
    text = re.sub(
        r"Confira os vídeos mais assistidos do \w+",
        "",
        text,
        flags=re.IGNORECASE,
    )
    text = re.sub(
        r"O g1 agora está no Telegram; Clique aqui para receber notícias diretamente no seu celular",  # noqa: E501
        "",
        text,
        flags=re.IGNORECASE,
    )

    text = html.unescape(text)
    text = unidecode(text)
    text = text.lower()
    text = text.replace("\r", " ").replace("\n", " ").replace("\t", " ")
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"[^a-zA-ZÀ-ÿ\s]", " ", text)
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)
    text = re.sub(r"#\w+", " ", text)
    text = re.sub(r"@\w+", " ", text)
    text = re.sub(r"", "", text)
    text = re.sub(r"\s+", " ", text)

    emoji_pattern = re.compile(
        "["
        "\U0001f600-\U0001f64f"  # Emoticons
        "\U0001f300-\U0001f5ff"  # Símbolos e pictogramas
        "\U0001f680-\U0001f6ff"  # Símbolos de transporte e mapas
        "\U0001f1e0-\U0001f1ff"  # Bandeiras (alfabeto regional)
        "\U00002500-\U00002bef"  # Setas e símbolos variados
        "\U00002702-\U000027b0"  # Outros tipos de símbolos e pictogramas
        "\U000024c2-\U0001f251"  # Diversos símbolos extras
        "\U0001f900-\U0001f9ff"  # Símbolos e pictogramas de extensão
        "\U0001fa70-\U0001faff"  # Novos pictogramas e símbolos adicionais
        "\U00002600-\U000026ff"  # Diversos símbolos como ☀️, ⛅, ☁️
        "\U0001f7e0-\U0001f7ff"  # Quadrados coloridos, etc.
        "\U0001f780-\U0001f7d8"  # Círculos e quadrados coloridos
        "]+",
        flags=re.UNICODE,
    )

    return emoji_pattern.sub(r"", text).strip()

    # - tokens = word_tokenize(text, language="portuguese")
    # - stemmer = RSLPStemmer()
    # - custom_stopwords = {"g1", "noticia", "telegram", "reproducao"}
    # - stop_words = set(stopwords.words("portuguese")).union(custom_stopwords)
    # - processed_tks =[stemmer.stem(word) for word in tokens if word not in stop_words]

In [ ]:
itens["body"] = itens["body"].apply(preprocess_text)
itens["title"] = itens["title"].apply(preprocess_text)
itens["caption"] = itens["caption"].apply(preprocess_text)

itens.head(2)

In [ ]:
def preprocess_title_caption(text: str) -> str:
    """Preprocess title and caption text."""
    if pd.isna(text):
        return ""

    text = html.unescape(text)
    text = unidecode(text)
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"[^a-zA-ZÀ-ÿ\s]", " ", text)
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text.lower()

In [ ]:
def preprocess_text(row: pd.Series) -> str:
    """Join all texts into one."""
    return f"{' '.join(row['tags'])} {row['title']} {row['caption']} {row['body']}"

In [5]:
itens["full_text"] = itens.apply(preprocess_text, axis=1)

In [6]:
itens.head(1)

,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay,processed_body,full_text
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,caso bruno e dom suspeito tem prisao temporari...,"Após audiência de custódia, a Justiça do Amazo...",jeferson da silva lima foi escoltado por agent...,['am' 'amazonas' 'noticia'],57,3,10875.0,1.386294,4.280184,0.247584,apo audienc custod jus ama decret tard dest sa...,am amazonas noticia caso bruno e dom suspeito ...


In [7]:
itens.to_parquet(
    "../data/interim/itens_popularity_body_processed.parquet", engine="pyarrow"
)

Aplicando embeddings

In [66]:
from pyarrow.parquet import read_table
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize

/workspaces/fiap-mlet-datathon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
path = "../data/interim/itens_popularity_body_processed.parquet"

tab = read_table(path)
itens = tab.to_pandas(types_mapper=pd.ArrowDtype, ignore_metadata=True)
itens

/workspaces/fiap-mlet-datathon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay,full_text
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,caso bruno e dom o suspeito tem prisao tempora...,apos audiencia de custodia a justica do amazon...,jeferson da silva lima foi escoltado por agent...,['am' 'amazonas' 'noticia'],57,3,10875.0,1.386294,4.280184,0.247584,am amazonas noticia caso bruno e dom o suspeit...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,linguajar dos santarenos e diferenciado e chei...,vista aerea de santarem adrio denner ad produc...,as expressoes santarenas nao significam apenas...,['pa' 'santarem-regiao' 'noticia'],1151,2,15000.0,1.098612,3.845143,0.0,pa santarem-regiao noticia linguajar dos santa...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,ex premie shinzo abe morre apos ser baleado no...,novo video mostra que assassino de shinzo abe ...,ex primeiro ministro foi atingido por tiros de...,['mundo' 'noticia'],37,12498,60000.0,9.433404,75.467231,11.866254,mundo noticia ex premie shinzo abe morre apos ...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,relator no stf fachin vota contra marco tempor...,relator no stf fachin vota contra marco tempor...,ministro defendeu que posse indigena e diferen...,['politica' 'noticia'],339,3,35856.5,1.386294,7.743355,0.0,politica noticia relator no stf fachin vota co...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,apos votos pedido de vista suspende julgamento...,apos um pedido de vista mais tempo para analis...,pelo marco temporal indios so podem reivindica...,['politica' 'noticia'],333,6,41051.25,1.94591,11.880025,0.000001,politica noticia apos votos pedido de vista su...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255598,aca76b7b-9c02-4070-a4a9-00f8c9b4dae2,http://g1.globo.com/politica/eleicoes/2022/not...,2022-04-18 23:32:54+00:00,2022-05-02 10:53:35+00:00,eleicoes titulo de eleitor cancelado o que faz...,detalhe da urna eletronica e a tecla confirma ...,cidadao pode consultar a situacao do titulo de...,['politica' 'eleicoes-2022' 'noticia'],118,3,50000.0,1.386294,9.704061,0.026584,politica eleicoes-2022 noticia eleicoes titulo...
255599,20de2172-b557-4881-8c55-2706db029e9b,http://g1.globo.com/politica/eleicoes/2022/not...,2022-08-09 00:53:27+00:00,2022-09-19 20:30:25+00:00,plano de governo ciro gomes pdt,candidato do pdt a presidencia da republica ci...,em documento de paginas candidato a presidenci...,['politica' 'eleicoes-2022' 'noticia'],6,155,47431.0,5.049856,34.051684,25.226108,politica eleicoes-2022 noticia plano de govern...
255600,347c7e3d-ff53-4ba0-8de7-8bb860b04a88,http://g1.globo.com/politica/eleicoes/2022/not...,2022-03-15 18:18:54+00:00,2022-03-15 18:26:34+00:00,moro ve economia brasileira estagnada e diz qu...,o pre candidato do podemos a presidencia da re...,ex ministro da justica de bolsonaro e pre cand...,['politica' 'eleicoes-2022' 'noticia'],152,1,9409.0,0.693147,2.038477,0.00102,politica eleicoes-2022 noticia moro ve economi...
255601,26f7ff70-46da-42d2-9354-578e8f828a16,http://g1.globo.com/politica/eleicoes/2022/not...,2022-03-23 15:13:50+00:00,2022-03-24 00:54:34+00:00,lula e hoje aquele que melhor reflete o sentim...,ex governador de sp geraldo alckmin se filia a...,ex governador de sp deu a declaracao durante a...,['politica' 'eleicoes-2022' 'noticia'],144,4,20027.25,1.609438,6.442137,0.00481,politica eleicoes-2022 noticia lula e hoje aqu...


In [4]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [5]:
def generate_embeddings(texts: list, model: SentenceTransformer):
    """Create embeddings for all texts."""
    return model.encode(texts, show_progress_bar=True)

In [6]:
embeddings = generate_embeddings(itens["full_text"].tolist(), model)

Batches: 100%|██████████| 7988/7988 [1:04:46<00:00,  2.06it/s]


In [ ]:
np.save("../data/interim/embeddings.npy", embeddings)

In [64]:
embeddings = np.load("../data/interim/embeddings.npy")

In [67]:
embeddings = normalize(embeddings)

In [68]:
news_embedding_dict = dict(zip(itens["page"], embeddings))

In [69]:
import pickle

with Path("../data/processed/news_embedding.pkl").open("wb") as f:
    pickle.dump(news_embedding_dict, f)

In [70]:
import pickle

with Path("../data/processed/news_embedding.pkl").open("rb") as f:
    news_embedding_dict = pickle.load(f)  # noqa: S301

Testando similaridade

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
def recommend_similar_news(
    news_id: str, news_embedding_dict: dict, itens: pd.DataFrame, top_n: int = 5
) -> list[dict]:
    """Recomenda notícias similares com base na similaridade de cosseno."""
    target_embedding = news_embedding_dict[news_id].reshape(1, -1)

    all_ids = list(news_embedding_dict.keys())
    all_embeddings = np.array([news_embedding_dict[i] for i in all_ids])
    similarities = cosine_similarity(target_embedding, all_embeddings).flatten()

    similar_indices = np.argsort(similarities)[::-1][1 : top_n + 1]
    return [
        {
            "page": all_ids[i],
            "title": itens.loc[itens["page"] == all_ids[i], "title"].to_numpy()[0],  # type: ignore[attr-defined]
            "similarity": similarities[i],
        }
        for i in similar_indices
    ]

Nos testes abaixo, vemos que está funcionando bem:
- Todas as notícias previstas tem envolvimento com o caso "Bruno e Dom" (que é o tema principal da notícia de entrada)
- As quatro primeiras notícias, que mais citam o caso, acabam aparecendo primeiro (score mais alto)
- Através de refinamentos no pré-processamento dos textos, conseguimos chegar em um resultado satisfatório

Por conta disso, vemos que a recomendação de notícias está funcionando bem.

In [14]:
recommend_similar_news(
    "13db0ab1-eea2-4603-84c4-f40a876c7400", news_embedding_dict, itens
)

[{'page': 'b5795aea-4807-49a2-b109-5a5e9ff6c572',
  'title': 'terceiro suspeito de envolvimento nas mortes de bruno e dom e preso no am',
  'similarity': np.float32(0.92192894)},
 {'page': '41eb53f7-9bcc-4e3f-b5ff-547ff998fe60',
  'title': 'justica decreta prisao de terceiro suspeito por envolvimento na morte de bruno pereira e dom phillips no am',
  'similarity': np.float32(0.89857346)},
 {'page': 'c04e4430-e26c-4799-b596-22114c882d78',
  'title': 'policia ouviu esposa de pelado suspeito investigado por desaparecimentos de bruno pereira e dom phillips',
  'similarity': np.float32(0.8413029)},
 {'page': '3fdb8325-614d-4e65-ad8b-5518bc35204a',
  'title': 'desaparecidos na amazonia pericia encontra sangue em lancha e delegado pede prisao temporaria de suspeito',
  'similarity': np.float32(0.8322212)},
 {'page': 'aeeaae6c-c771-4ad4-9613-0c905f8ca603',
  'title': 'defensoria diz que desaparecimento de jornalista e indigenista no amazonas revela completa omissao na defesa dos povos indigena

In [22]:
itens.head(1)

,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay,processed_body,full_text
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,caso bruno e dom suspeito tem prisao temporari...,"Após audiência de custódia, a Justiça do Amazo...",jeferson da silva lima foi escoltado por agent...,['am' 'amazonas' 'noticia'],57,3,10875.0,1.386294,4.280184,0.247584,apo audienc custod jus ama decret tard dest sa...,caso bruno e dom suspeito tem prisao temporari...


Agora que já fizemos o processamento necessário para construir a similaridade de notícias, vamos fazer o mesmo para construir a similaridade de usuários.

A ideia é encontrar usuários com perfis similares, levando em consideração suas métricas em cada notícia que ele acessou. O motivo disso é simples: queremos recomendar coisas que o usuário pode gostar mais e, para isso, podemos recomendar aquilo que usuários similares a ele gostam também.

In [4]:
def read_csv_pyarrow(path: Path) -> pd.DataFrame:
    """Read a CSV file with PyArrow as backend.

    Args:
        path (Path): the CSV file path

    Returns:
        pd.DataFrame: a Pandas DataFrame

    """
    try:
        return pd.read_csv(
            path,
            engine="pyarrow",
            dtype_backend="pyarrow",
        )
    except pd.errors.ParserError:
        return pv.read_csv(
            path,
            parse_options=pv.ParseOptions(newlines_in_values=True),
        ).to_pandas(types_mapper=pd.ArrowDtype)


train_path = "../data/raw/files/treino/*.csv"
train_files = Path.cwd().glob(train_path)

train = pd.concat(
    (read_csv_pyarrow(file) for file in train_files),
    ignore_index=True,
)
list_columns = [
    "history",
    "timestampHistory",
    "numberOfClicksHistory",
    "timeOnPageHistory",
    "scrollPercentageHistory",
    "pageVisitsCountHistory",
]

train = train.drop(columns=["timestampHistory_new"])
train[list_columns] = train[list_columns].apply(lambda x: x.str.split(", "))
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577942 entries, 0 to 577941
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype                      
---  ------                   --------------   -----                      
 0   userId                   577942 non-null  string[pyarrow]            
 1   userType                 577942 non-null  string[pyarrow]            
 2   historySize              577942 non-null  int64[pyarrow]             
 3   history                  577942 non-null  list<item: string>[pyarrow]
 4   timestampHistory         577942 non-null  list<item: string>[pyarrow]
 5   numberOfClicksHistory    577942 non-null  list<item: string>[pyarrow]
 6   timeOnPageHistory        577942 non-null  list<item: string>[pyarrow]
 7   scrollPercentageHistory  577942 non-null  list<item: string>[pyarrow]
 8   pageVisitsCountHistory   577942 non-null  list<item: string>[pyarrow]
dtypes: int64[pyarrow](1), list<item: string>[pyarrow](6), string

In [46]:
def preprocess_users(users_df):
    unique_news = {news for history in users_df["history"] for news in history}
    news_index = {news: idx for idx, news in enumerate(unique_news)}
    user_index = {row["userId"]: idx for idx, row in users_df.iterrows()}
    return news_index, user_index


# Função para criar vetores de interação do usuário com as notícias
def build_user_vectors(users_df, news_index):
    user_vectors = dok_matrix((len(users_df), len(news_index)), dtype=np.float32)

    for user_idx, row in users_df.iterrows():
        for news_id, visits, time_spent, scroll in zip(
            row["history"],
            row["pageVisitsCountHistory"],
            row["timeOnPageHistory"],
            row["scrollPercentageHistory"],
        ):
            interaction_score = (
                np.clip(pd.to_numeric(visits), 1, 10)
                + (np.clip(pd.to_numeric(time_spent), 0, 300000) / 1000)
                + (np.clip(pd.to_numeric(scroll), 0, 100) / 100)
            )
            user_vectors[user_idx, news_index[news_id]] = interaction_score

    return user_vectors.tocsr()


# Função otimizada para calcular similaridade entre usuários por blocos
def compute_user_similarity(user_vectors, top_k=10, batch_size=100):
    top_similarities = {}
    num_users = user_vectors.shape[0]

    for start in range(0, num_users, batch_size):
        end = min(start + batch_size, num_users)
        batch_sim = cosine_similarity(
            user_vectors[start:end], user_vectors, dense_output=False
        )

        for i in range(batch_sim.shape[0]):
            user_idx = start + i
            similar_users = list(zip(batch_sim[i].indices, batch_sim[i].data))
            similar_users = sorted(similar_users, key=lambda x: x[1], reverse=True)[
                :top_k
            ]
            top_similarities[user_idx] = similar_users

    return top_similarities


# Criar dicionário de similaridade de usuários de forma otimizada
def create_user_similarity_dict(users_df, top_similarities):
    user_similarity_dict = {}
    user_ids = users_df["userId"].tolist()

    for idx, similar_users in top_similarities.items():
        user_history_size = users_df.iloc[idx]["historySize"]
        small_history_size = 5
        if user_history_size <= small_history_size:
            continue

        filtered_similar_users = [
            (user_ids[j], score)
            for j, score in similar_users
            if j < len(user_ids)
            and users_df.iloc[j]["historySize"] > small_history_size
        ]

        if filtered_similar_users:
            user_similarity_dict[user_ids[idx]] = filtered_similar_users

    return user_similarity_dict


# Função para recomendar usuários similares
def recommend_similar_users(user_id, user_similarity_dict, top_n=5):
    return user_similarity_dict.get(user_id, [])[1 : top_n + 1]

In [7]:
news_index, user_index = preprocess_users(train)

In [8]:
user_vectors = build_user_vectors(train, news_index)

In [10]:
import pickle

with Path("../data/processed/user_vectors.pkl").open("wb") as f:
    pickle.dump(user_vectors, f)

In [11]:
top_similarities = compute_user_similarity(user_vectors)

In [12]:
with Path("../data/processed/top_similarities.pkl").open("wb") as f:
    pickle.dump(top_similarities, f)

In [36]:
user_similarity_dict = create_user_similarity_dict(train, top_similarities)

In [37]:
with Path("../data/processed/user_similarity_dict.pkl").open("wb") as f:
    pickle.dump(user_similarity_dict, f)

É difícil testar a recomendação de usuários similares, pois é um cálculo complexo que depende do engajamento do usuário nas notícias e nas notícias que ele visualizou. Além disso, da forma que foi construído, nosso objetivo é que o grupo de usuários tenha baixa influência no resultado final, então limitamos bastante seus resultados:

1. Só irá recomendar usuários com um histórico maior que 5
2. Só irá recomendar até 5 usuários similares
3. Para usuários sem muita similaridade, ele irá retornar uma lista vazia

In [48]:
recommend_similar_users(
    "0f95a7b52ac0d6daee1d27babc67d07ffd85c4d9b13f901b5abb49a92af2597c",
    user_similarity_dict,
    top_n=5,
)

[('e9b0d09c113b07ef40f3463e76c2ad28b11f4bb3b9506c92bc6a209797ac9327',
  np.float32(0.20362407))]

In [49]:
train.query(
    """userId == '0f95a7b52ac0d6daee1d27babc67d07ffd85c4d9b13f901b5abb49a92af2597c'\
    or userId == 'e9b0d09c113b07ef40f3463e76c2ad28b11f4bb3b9506c92bc6a209797ac9327'"""
)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
300014,0f95a7b52ac0d6daee1d27babc67d07ffd85c4d9b13f90...,Non-Logged,217,['52b4895d-0327-4ed9-9eef-7b48bc25e004' '6c92...,['1656694618153' '1656694655861' '165669473418...,['3' '2' '1' '0' '1' '0' '0' '0' '0' '1' '3' '...,['68176' '53328' '80169' '51415' '73739' '5140...,['32.95' '32.47' '62.87' '36.61' '71.07' '13.4...,['2' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '...
343719,e9b0d09c113b07ef40f3463e76c2ad28b11f4bb3b9506c...,Non-Logged,84,['7caf0079-a39e-4f7d-9d5b-7f8d8a625f2f' '38e5...,['1656696045359' '1656696072025' '165669608106...,['3' '0' '2' '0' '1' '1' '0' '0' '1' '0' '1' '...,['110000' '39138' '120000' '10000' '21450' '52...,['54.43' '46.87' '63.56' '18.45' '38.3' '43.32...,['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '...


In [ ]:
def recommend_news(
    user_id,
    timestamp,
    users_df,
    itens,
    news_embedding_dict,
    user_similarity_dict,
    top_n=5,
):
    user_row = users_df[users_df["userId"] == user_id].iloc[0]
    user_history = user_row["history"]
    history_timestamps = user_row["timestampHistory"]
    history_size = user_row["historySize"]

    last_timestamp = max(history_timestamps)
    time_diff = timestamp - int(last_timestamp)

    # ao acessar a prox. notícia em menos de um dia, o peso da similaridade aumenta
    # e o peso da popularidade diminui. Caso contrário, a popularidade importa mais
    # weight_popularity = 0.4 if time_diff < 86400 else 0.6
    # weight_news_similarity = 0.7 if time_diff < 86400 else 0.4
    # usuários com um histórico grande podem ter um perfil mais associado a outro
    # weight_user_similarity = 0.4 if history_size >= 10 else 0.1

    news_scores = defaultdict(float)
    for news_id in user_history:
        similar_news = recommend_similar_news(
            news_id, news_embedding_dict, itens, top_n=5
        )
        for similar in similar_news:
            news_scores[similar["page"]] += similar[
                "similarity"
            ]  # * weight_news_similarity

    similar_users = recommend_similar_users(user_id, user_similarity_dict, top_n=5)
    for similar_user, score in similar_users:
        similar_user_row = users_df[users_df["userId"] == similar_user].iloc[0]
        for news_id in similar_user_row["history"]:
            news_scores[news_id] += score  # * weight_user_similarity

    display(
        f"Antes do popularity: {sorted(news_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]}"
    )

    for _, row in itens.iterrows():
        news_scores[row["page"]] += row["popularity_score_decay"]  # * weight_popularity

    recommended_news = sorted(news_scores.items(), key=lambda x: x[1], reverse=True)
    return recommended_news[:top_n]

In [112]:
pd.to_datetime(1660533136590, unit="ms")

Timestamp('2022-08-15 03:12:16.590000')

In [92]:
validation_path = Path("../data/raw/validacao.csv")
validation = read_csv_pyarrow(validation_path)
validation.head(2)

list_columns = ["history", "timestampHistory"]

validation["history"] = (
    validation["history"]
    .str.replace(r"[\[\]']", "", regex=True)
    .str.replace(r"\n", "", regex=True)
    .str.split()
)
validation["timestampHistory"] = (
    validation["timestampHistory"].str.replace(r"[\[\]]", "", regex=True).str.split()
)
validation.head(1)

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,['1660533136590' '1660672113513']


In [134]:
for _, row in validation.head(3).iterrows():
    for page, timestamp in zip(row["history"], row["timestampHistory"]):
        recommended = recommend_news(
            row["userId"],
            pd.to_numeric(timestamp),
            train,
            itens,
            news_embedding_dict,
            user_similarity_dict,
        )

        display(
            f"{row['userId']} - {page} e {timestamp} ||| Recomendamos {recommended}"
        )

        if page in [i[0] for i in recommended]:
            display(f"Acertou para o user: {row['userId']} e {timestamp}")

"Antes do popularity: [('64ba811e-705f-4a14-8152-69b9825bca10', np.float32(0.9250973)), ('7e90c659-e11b-4c86-9a39-4a31ed90fc4e', np.float32(0.9020767)), ('6c517284-c766-4c7e-a27f-09b4bc6ed8ab', np.float32(0.89715767)), ('be4a3141-72ce-4286-a424-84622cc4225c', np.float32(0.8940736)), ('a80614bb-c02c-4837-96ff-cb9239189dcd', np.float32(0.89312935))]"

"e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9 - be89a7da-d9fa-49d4-9fdc-388c27a15bc8 e 1660533136590 ||| Recomendamos [('d730c4a6-e8f6-4fde-b73a-afbe148479cd', 74.56942808778152), ('1f2b9c2f-a2d2-4192-b009-09065da8ec23', 71.56161185107858), ('598ed114-fd5a-4d82-90d8-f1e893cb0892', 70.04208732225547), ('d173164f-dcfc-4e32-8006-64ab44a57c66', 67.72388146759798), ('279ccbff-f203-4c6d-aa48-83d97f085302', 67.70989781133794)]"

"Antes do popularity: [('64ba811e-705f-4a14-8152-69b9825bca10', np.float32(0.9250973)), ('7e90c659-e11b-4c86-9a39-4a31ed90fc4e', np.float32(0.9020767)), ('6c517284-c766-4c7e-a27f-09b4bc6ed8ab', np.float32(0.89715767)), ('be4a3141-72ce-4286-a424-84622cc4225c', np.float32(0.8940736)), ('a80614bb-c02c-4837-96ff-cb9239189dcd', np.float32(0.89312935))]"

"e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9 - 01c59ff6-fb82-4258-918f-2910cb2d4c52 e 1660672113513 ||| Recomendamos [('d730c4a6-e8f6-4fde-b73a-afbe148479cd', 74.56942808778152), ('1f2b9c2f-a2d2-4192-b009-09065da8ec23', 71.56161185107858), ('598ed114-fd5a-4d82-90d8-f1e893cb0892', 70.04208732225547), ('d173164f-dcfc-4e32-8006-64ab44a57c66', 67.72388146759798), ('279ccbff-f203-4c6d-aa48-83d97f085302', 67.70989781133794)]"

"Antes do popularity: [('3dd46fc3-7740-493d-b31f-cb9fa270dd97', np.float32(0.82783496)), ('59b6746f-6005-4745-8cc8-f6592e1a6695', np.float32(0.8205815)), ('640494af-a858-4e85-98cd-cb485b86a7b1', np.float32(0.76791286)), ('71905503-b665-4e74-b00b-d89762d02393', np.float32(0.7532753)), ('9032b8a1-cb00-4a17-ae27-fc6cd82715f2', np.float32(0.74598706))]"

"d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdeee870d49b897e1e99cd - 77901133-aee7-4f7b-afc0-652231d76fe9 e 1660556860253 ||| Recomendamos [('d730c4a6-e8f6-4fde-b73a-afbe148479cd', 74.56942808778152), ('1f2b9c2f-a2d2-4192-b009-09065da8ec23', 71.56161185107858), ('598ed114-fd5a-4d82-90d8-f1e893cb0892', 70.04208732225547), ('d173164f-dcfc-4e32-8006-64ab44a57c66', 67.72388146759798), ('279ccbff-f203-4c6d-aa48-83d97f085302', 67.70989781133794)]"

"Antes do popularity: [('a4f87cfe-57af-49a1-ae56-704590044419', np.float32(0.8670955)), ('025244d6-873f-4b3d-80ea-a0636c89b4ea', np.float32(0.8184153)), ('d8959312-e55a-4a41-b46c-68b3daee9498', np.float32(0.8110373)), ('901b9833-c7ad-45c9-bcbb-8afacd5cd7c4', np.float32(0.8040944)), ('d42fdc2f-3ce4-494b-be1c-6b68cd6fd2ba', np.float32(0.8028907))]"

"755062dd39a48809880cf363b04268c3af2c003088cde02636fcca973a3564d1 - 857aa90f-a7ec-410d-ba82-dfa4f85d4e71 e 1660561649242 ||| Recomendamos [('d730c4a6-e8f6-4fde-b73a-afbe148479cd', 74.56942808778152), ('1f2b9c2f-a2d2-4192-b009-09065da8ec23', 71.56161185107858), ('598ed114-fd5a-4d82-90d8-f1e893cb0892', 70.04208732225547), ('d173164f-dcfc-4e32-8006-64ab44a57c66', 67.72388146759798), ('279ccbff-f203-4c6d-aa48-83d97f085302', 67.70989781133794)]"

In [132]:
recommend_news(
    "d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdeee870d49b897e1e99cd",
    1660556860253,
    train,
    itens,
    news_embedding_dict,
    user_similarity_dict,
)

"Antes do popularity: [('3dd46fc3-7740-493d-b31f-cb9fa270dd97', np.float32(0.82783496)), ('59b6746f-6005-4745-8cc8-f6592e1a6695', np.float32(0.8205815)), ('640494af-a858-4e85-98cd-cb485b86a7b1', np.float32(0.76791286)), ('71905503-b665-4e74-b00b-d89762d02393', np.float32(0.7532753)), ('9032b8a1-cb00-4a17-ae27-fc6cd82715f2', np.float32(0.74598706))]"

[('d730c4a6-e8f6-4fde-b73a-afbe148479cd', 74.56942808778152),
 ('1f2b9c2f-a2d2-4192-b009-09065da8ec23', 71.56161185107858),
 ('598ed114-fd5a-4d82-90d8-f1e893cb0892', 70.04208732225547),
 ('d173164f-dcfc-4e32-8006-64ab44a57c66', 67.72388146759798),
 ('279ccbff-f203-4c6d-aa48-83d97f085302', 67.70989781133794)]

In [87]:
train.query(
    "userId == 'e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9'"
)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
482774,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,6,['c7b6f56c-3304-4fe5-994a-50c8937d6431' 'c6af...,['1657003078783' '1659484116867' '165970076624...,['5' '1' '10' '16' '4' '2'],['20848' '10000' '37197' '90813' '23755' '1000...,['36.51' '19.78' '46.59' '36.45' '32.46' '13.89'],['1' '1' '1' '1' '1' '1']


In [114]:
itens.query(
    """page == 'd730c4a6-e8f6-4fde-b73a-afbe148479cd'\
    or page == '1f2b9c2f-a2d2-4192-b009-09065da8ec23'\
    or page == '598ed114-fd5a-4d82-90d8-f1e893cb0892'\
    or page == 'd173164f-dcfc-4e32-8006-64ab44a57c66'\
    or page == '279ccbff-f203-4c6d-aa48-83d97f085302'\
    """
)

,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay
71096,d730c4a6-e8f6-4fde-b73a-afbe148479cd,http://g1.globo.com/sp/santos-regiao/noticia/2...,2022-08-14 09:49:40+00:00,2022-08-15 12:41:00+00:00,Mulher é vítima de racismo em shopping no lito...,Vítima relata ofensas racistas em shopping no ...,Filha da vítima contou ao g1 que a mãe está em...,['sp' 'santos-regiao' 'noticia'],0,4533,68569.0,8.41936,74.569428,74.569428
99533,598ed114-fd5a-4d82-90d8-f1e893cb0892,http://g1.globo.com/sp/santos-regiao/noticia/2...,2022-08-11 07:51:13+00:00,2022-08-11 19:16:38+00:00,Mãe descobre paradeiro de filha desaparecida p...,"Karla (à esq) e Marcela, irmãs vão se reencont...","Mulher que mora no Morro da Nova Cintra, em Sa...",['sp' 'santos-regiao' 'noticia'],3,8449,70000.0,9.041922,81.377295,70.042087
120511,d173164f-dcfc-4e32-8006-64ab44a57c66,http://g1.globo.com/rj/rio-de-janeiro/noticia/...,2022-08-11 14:04:19+00:00,2022-08-15 19:13:50+00:00,"Golpe em idosa: ‘Mata essa velha!’, mandou Ro...",Sabine Boghici Reprodução Rosa Stanesco Nicola...,As namoradas e outras duas pessoas foram presa...,['rj' 'rio-de-janeiro' 'noticia'],3,7318,68426.5,8.898229,78.683925,67.723881
171665,1f2b9c2f-a2d2-4192-b009-09065da8ec23,http://g1.globo.com/rj/rio-de-janeiro/noticia/...,2022-08-12 09:49:53+00:00,2022-08-12 13:07:13+00:00,"VÍDEO: ‘Me ajuda, por favor! Não! Socorro!’, s...","'Me ajuda, por favor!', suplica influenciadora...","Namorado afirmava que, se ela fugisse, ele iri...",['rj' 'rio-de-janeiro' 'noticia'],2,9744,66110.0,9.18451,79.087812,71.561612
205507,279ccbff-f203-4c6d-aa48-83d97f085302,http://g1.globo.com/sp/sao-paulo/noticia/2022/...,2022-08-13 22:51:15+00:00,2022-08-15 01:05:01+00:00,Policial que matou lutador Leandro Lo foi a bo...,Policial que matou lutador Leandro Lo foi a bo...,Henrique Veloso saiu de boate com garota de pr...,['sp' 'sao-paulo' 'noticia'],1,7314,60000.0,8.897682,71.181459,67.709898


In [90]:
itens.query(
    "page == 'be89a7da-d9fa-49d4-9fdc-388c27a15bc8' or page == '01c59ff6-fb82-4258-918f-2910cb2d4c52'"
)

,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay
4506,01c59ff6-fb82-4258-918f-2910cb2d4c52,http://g1.globo.com/saude/variola-dos-macacos/...,2022-08-13 16:56:59+00:00,2022-08-13 16:57:00+00:00,"Varíola dos macacos: 2,9 mil casos são investi...",UFG desenvolve teste rápido e barato para dete...,Brasil ainda está construindo rede e aparatos ...,['saude' 'variola-dos-macacos' 'noticia'],1,574,60000.0,6.35437,50.83496,48.35571
136476,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-08-14 22:43:50+00:00,2022-08-14 22:50:50+00:00,VÍDEO: vestido de aniversariante pega fogo em ...,Vestido de aniversariante pega fogo em festa d...,"Apesar do susto, Bianca Macanoni contou ao g1...",['am' 'amazonas' 'noticia'],0,989,61645.0,6.897705,56.316312,56.316312
